In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
#     eloss += tf.reduce_mean(targetQs) # minimize Q
#     aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
#     eloss += tf.reduce_mean(eQs_logits) # minimize Q
#     aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.7005 eloss:2.5676 aloss2:2.0114 exploreP:0.9984
Episode:1 meanR:19.5000 R:23.0000 rate:0.0460 aloss:0.6984 eloss:2.5581 aloss2:2.0186 exploreP:0.9961
Episode:2 meanR:19.3333 R:19.0000 rate:0.0380 aloss:0.6981 eloss:2.5169 aloss2:2.0320 exploreP:0.9943
Episode:3 meanR:21.5000 R:28.0000 rate:0.0560 aloss:0.6941 eloss:2.5139 aloss2:2.0377 exploreP:0.9915
Episode:4 meanR:21.8000 R:23.0000 rate:0.0460 aloss:0.6980 eloss:2.4980 aloss2:2.0438 exploreP:0.9893
Episode:5 meanR:20.5000 R:14.0000 rate:0.0280 aloss:0.6987 eloss:2.5075 aloss2:2.0360 exploreP:0.9879
Episode:6 meanR:20.1429 R:18.0000 rate:0.0360 aloss:0.6938 eloss:2.4371 aloss2:2.0670 exploreP:0.9861
Episode:7 meanR:20.1250 R:20.0000 rate:0.0400 aloss:0.6945 eloss:2.4760 aloss2:2.0454 exploreP:0.9842
Episode:8 meanR:23.4444 R:50.0000 rate:0.1000 aloss:0.6965 eloss:2.3682 aloss2:2.0960 exploreP:0.9793
Episode:9 meanR:23.2000 R:21.0000 rate:0.0420 aloss:0.6954 eloss:2.3518 aloss2:2.1

Episode:80 meanR:26.3704 R:13.0000 rate:0.0260 aloss:0.7009 eloss:1.2518 aloss2:2.9898 exploreP:0.8096
Episode:81 meanR:26.4512 R:33.0000 rate:0.0660 aloss:0.6882 eloss:1.2526 aloss2:3.0003 exploreP:0.8070
Episode:82 meanR:26.4217 R:24.0000 rate:0.0480 aloss:0.6843 eloss:1.2394 aloss2:3.0300 exploreP:0.8050
Episode:83 meanR:26.4286 R:27.0000 rate:0.0540 aloss:0.7024 eloss:1.2384 aloss2:3.0157 exploreP:0.8029
Episode:84 meanR:26.3294 R:18.0000 rate:0.0360 aloss:0.6825 eloss:1.2200 aloss2:3.0638 exploreP:0.8015
Episode:85 meanR:26.5233 R:43.0000 rate:0.0860 aloss:0.6954 eloss:1.2203 aloss2:3.0601 exploreP:0.7981
Episode:86 meanR:26.3218 R:9.0000 rate:0.0180 aloss:0.7325 eloss:1.2034 aloss2:3.0776 exploreP:0.7974
Episode:87 meanR:26.2386 R:19.0000 rate:0.0380 aloss:0.7683 eloss:1.2071 aloss2:3.0927 exploreP:0.7959
Episode:88 meanR:26.6180 R:60.0000 rate:0.1200 aloss:0.6924 eloss:1.1856 aloss2:3.1217 exploreP:0.7912
Episode:89 meanR:26.6889 R:33.0000 rate:0.0660 aloss:0.7071 eloss:1.1655 a

Episode:159 meanR:37.8600 R:55.0000 rate:0.1100 aloss:0.6821 eloss:0.6007 aloss2:5.7151 exploreP:0.5946
Episode:160 meanR:37.5800 R:18.0000 rate:0.0360 aloss:0.6815 eloss:0.5814 aloss2:5.6095 exploreP:0.5935
Episode:161 meanR:37.5900 R:50.0000 rate:0.1000 aloss:0.6889 eloss:0.6053 aloss2:5.7297 exploreP:0.5906
Episode:162 meanR:37.3500 R:37.0000 rate:0.0740 aloss:0.6705 eloss:0.5997 aloss2:5.8562 exploreP:0.5885
Episode:163 meanR:37.4800 R:39.0000 rate:0.0780 aloss:0.6767 eloss:0.5953 aloss2:5.8946 exploreP:0.5862
Episode:164 meanR:37.8400 R:58.0000 rate:0.1160 aloss:0.6819 eloss:0.5933 aloss2:5.9002 exploreP:0.5829
Episode:165 meanR:38.1000 R:47.0000 rate:0.0940 aloss:0.6739 eloss:0.5873 aloss2:6.0027 exploreP:0.5802
Episode:166 meanR:38.2600 R:80.0000 rate:0.1600 aloss:0.6806 eloss:0.5924 aloss2:6.0268 exploreP:0.5757
Episode:167 meanR:38.1200 R:10.0000 rate:0.0200 aloss:0.6711 eloss:0.5682 aloss2:5.9383 exploreP:0.5751
Episode:168 meanR:39.0600 R:125.0000 rate:0.2500 aloss:0.6749 el

Episode:238 meanR:31.9900 R:10.0000 rate:0.0200 aloss:0.7405 eloss:0.5791 aloss2:6.9335 exploreP:0.4700
Episode:239 meanR:31.8100 R:12.0000 rate:0.0240 aloss:0.7333 eloss:0.6116 aloss2:6.8686 exploreP:0.4694
Episode:240 meanR:31.2600 R:12.0000 rate:0.0240 aloss:0.7313 eloss:0.6040 aloss2:6.8469 exploreP:0.4689
Episode:241 meanR:30.9200 R:11.0000 rate:0.0220 aloss:0.7315 eloss:0.4883 aloss2:6.9153 exploreP:0.4684
Episode:242 meanR:30.3600 R:13.0000 rate:0.0260 aloss:0.7311 eloss:0.4964 aloss2:6.8492 exploreP:0.4678
Episode:243 meanR:30.0900 R:11.0000 rate:0.0220 aloss:0.7459 eloss:0.5943 aloss2:6.8944 exploreP:0.4673
Episode:244 meanR:29.7400 R:10.0000 rate:0.0200 aloss:0.7518 eloss:0.5056 aloss2:6.8304 exploreP:0.4668
Episode:245 meanR:29.7300 R:10.0000 rate:0.0200 aloss:0.7444 eloss:0.5776 aloss2:6.7697 exploreP:0.4664
Episode:246 meanR:29.2100 R:9.0000 rate:0.0180 aloss:0.7482 eloss:0.6041 aloss2:6.8163 exploreP:0.4660
Episode:247 meanR:28.6000 R:10.0000 rate:0.0200 aloss:0.7465 elos

Episode:317 meanR:12.0100 R:12.0000 rate:0.0240 aloss:0.7591 eloss:0.6177 aloss2:6.8866 exploreP:0.4299
Episode:318 meanR:12.0400 R:16.0000 rate:0.0320 aloss:0.7587 eloss:0.5221 aloss2:6.9895 exploreP:0.4292
Episode:319 meanR:12.0500 R:11.0000 rate:0.0220 aloss:0.7421 eloss:0.5607 aloss2:6.9711 exploreP:0.4287
Episode:320 meanR:11.9800 R:12.0000 rate:0.0240 aloss:0.7555 eloss:0.5618 aloss2:6.9899 exploreP:0.4282
Episode:321 meanR:11.9000 R:9.0000 rate:0.0180 aloss:0.7531 eloss:0.4936 aloss2:7.0018 exploreP:0.4278
Episode:322 meanR:11.9300 R:14.0000 rate:0.0280 aloss:0.7410 eloss:0.5749 aloss2:6.9417 exploreP:0.4273
Episode:323 meanR:11.8900 R:10.0000 rate:0.0200 aloss:0.7505 eloss:0.4735 aloss2:6.9587 exploreP:0.4268
Episode:324 meanR:11.8600 R:10.0000 rate:0.0200 aloss:0.7468 eloss:0.5522 aloss2:7.0109 exploreP:0.4264
Episode:325 meanR:11.6200 R:10.0000 rate:0.0200 aloss:0.7572 eloss:0.5113 aloss2:6.9844 exploreP:0.4260
Episode:326 meanR:11.5400 R:9.0000 rate:0.0180 aloss:0.7354 eloss

...

Episode:633 meanR:11.0200 R:12.0000 rate:0.0240 aloss:0.7862 eloss:0.5377 aloss2:6.9397 exploreP:0.3046
Episode:634 meanR:11.0300 R:12.0000 rate:0.0240 aloss:0.7638 eloss:0.5474 aloss2:7.0560 exploreP:0.3043
Episode:635 meanR:11.0300 R:10.0000 rate:0.0200 aloss:0.7649 eloss:0.5324 aloss2:7.1288 exploreP:0.3040
Episode:636 meanR:11.0300 R:10.0000 rate:0.0200 aloss:0.8040 eloss:0.6315 aloss2:6.8163 exploreP:0.3037
Episode:637 meanR:11.0200 R:11.0000 rate:0.0220 aloss:0.7754 eloss:0.6642 aloss2:7.0842 exploreP:0.3034
Episode:638 meanR:11.0100 R:10.0000 rate:0.0200 aloss:0.7655 eloss:0.5363 aloss2:7.0443 exploreP:0.3031
Episode:639 meanR:11.0500 R:14.0000 rate:0.0280 aloss:0.7483 eloss:0.5077 aloss2:6.9741 exploreP:0.3027
Episode:640 meanR:11.0400 R:8.0000 rate:0.0160 aloss:0.7666 eloss:0.4740 aloss2:7.0507 exploreP:0.3024
Episode:641 meanR:11.0500 R:12.0000 rate:0.0240 aloss:0.7552 eloss:0.5923 aloss2:7.0027 exploreP:0.3021
Episode:642 meanR:11.1100 R:17.0000 rate:0.0340 aloss:0.7693 elos

Episode:712 meanR:10.8500 R:13.0000 rate:0.0260 aloss:0.7634 eloss:0.5337 aloss2:7.1804 exploreP:0.2804
Episode:713 meanR:10.8700 R:12.0000 rate:0.0240 aloss:0.7590 eloss:0.5187 aloss2:7.2955 exploreP:0.2801
Episode:714 meanR:10.8500 R:8.0000 rate:0.0160 aloss:0.7773 eloss:0.5549 aloss2:7.1651 exploreP:0.2799
Episode:715 meanR:10.8400 R:8.0000 rate:0.0160 aloss:0.7544 eloss:0.4576 aloss2:7.1610 exploreP:0.2796
Episode:716 meanR:10.8800 R:13.0000 rate:0.0260 aloss:0.7392 eloss:0.5224 aloss2:7.1224 exploreP:0.2793
Episode:717 meanR:10.8800 R:11.0000 rate:0.0220 aloss:0.7450 eloss:0.4486 aloss2:7.2181 exploreP:0.2790
Episode:718 meanR:10.8600 R:10.0000 rate:0.0200 aloss:0.7488 eloss:0.4687 aloss2:7.1007 exploreP:0.2787
Episode:719 meanR:10.9000 R:13.0000 rate:0.0260 aloss:0.7402 eloss:0.4679 aloss2:7.2188 exploreP:0.2784
Episode:720 meanR:10.8800 R:11.0000 rate:0.0220 aloss:0.7589 eloss:0.5146 aloss2:7.2362 exploreP:0.2781
Episode:721 meanR:10.8700 R:9.0000 rate:0.0180 aloss:0.7359 eloss:

Episode:791 meanR:10.7200 R:11.0000 rate:0.0220 aloss:0.7686 eloss:0.5321 aloss2:7.2772 exploreP:0.2583
Episode:792 meanR:10.7200 R:9.0000 rate:0.0180 aloss:0.7220 eloss:0.4130 aloss2:7.1529 exploreP:0.2580
Episode:793 meanR:10.7100 R:9.0000 rate:0.0180 aloss:0.7203 eloss:0.4418 aloss2:7.2012 exploreP:0.2578
Episode:794 meanR:10.7300 R:11.0000 rate:0.0220 aloss:0.7579 eloss:0.5134 aloss2:7.2344 exploreP:0.2575
Episode:795 meanR:10.7800 R:15.0000 rate:0.0300 aloss:0.7700 eloss:0.5169 aloss2:7.2487 exploreP:0.2572
Episode:796 meanR:10.7800 R:10.0000 rate:0.0200 aloss:0.7707 eloss:0.5530 aloss2:7.2162 exploreP:0.2569
Episode:797 meanR:10.7900 R:10.0000 rate:0.0200 aloss:0.7693 eloss:0.4964 aloss2:7.2245 exploreP:0.2567
Episode:798 meanR:10.8000 R:11.0000 rate:0.0220 aloss:0.7713 eloss:0.5243 aloss2:7.2058 exploreP:0.2564
Episode:799 meanR:10.8100 R:10.0000 rate:0.0200 aloss:0.7745 eloss:0.5075 aloss2:7.2121 exploreP:0.2562
Episode:800 meanR:10.7900 R:8.0000 rate:0.0160 aloss:0.7809 eloss:

Episode:870 meanR:11.3000 R:12.0000 rate:0.0240 aloss:0.7438 eloss:0.4211 aloss2:7.3230 exploreP:0.2372
Episode:871 meanR:11.2700 R:9.0000 rate:0.0180 aloss:0.7685 eloss:0.5397 aloss2:7.1586 exploreP:0.2370
Episode:872 meanR:11.2200 R:9.0000 rate:0.0180 aloss:0.7474 eloss:0.4700 aloss2:7.2174 exploreP:0.2368
Episode:873 meanR:11.2000 R:10.0000 rate:0.0200 aloss:0.7580 eloss:0.4099 aloss2:7.3152 exploreP:0.2365
Episode:874 meanR:11.2000 R:8.0000 rate:0.0160 aloss:0.7705 eloss:0.4784 aloss2:7.3339 exploreP:0.2364
Episode:875 meanR:11.1400 R:11.0000 rate:0.0220 aloss:0.7348 eloss:0.4196 aloss2:7.3761 exploreP:0.2361
Episode:876 meanR:11.1200 R:11.0000 rate:0.0220 aloss:0.7362 eloss:0.4993 aloss2:7.1708 exploreP:0.2359
Episode:877 meanR:11.1300 R:11.0000 rate:0.0220 aloss:0.7623 eloss:0.4362 aloss2:7.2673 exploreP:0.2356
Episode:878 meanR:11.1500 R:12.0000 rate:0.0240 aloss:0.7770 eloss:0.5245 aloss2:7.2642 exploreP:0.2353
Episode:879 meanR:11.1300 R:14.0000 rate:0.0280 aloss:0.7685 eloss:

Episode:950 meanR:10.9000 R:11.0000 rate:0.0220 aloss:0.7461 eloss:0.4613 aloss2:7.3953 exploreP:0.2188
Episode:951 meanR:10.9100 R:9.0000 rate:0.0180 aloss:0.7377 eloss:0.4791 aloss2:7.3466 exploreP:0.2186
Episode:952 meanR:10.7800 R:9.0000 rate:0.0180 aloss:0.7040 eloss:0.4835 aloss2:7.1335 exploreP:0.2184
Episode:953 meanR:10.7800 R:9.0000 rate:0.0180 aloss:0.7268 eloss:0.4344 aloss2:7.3535 exploreP:0.2182
Episode:954 meanR:10.7400 R:11.0000 rate:0.0220 aloss:0.7656 eloss:0.5163 aloss2:7.3293 exploreP:0.2180
Episode:955 meanR:10.7300 R:12.0000 rate:0.0240 aloss:0.7548 eloss:0.4761 aloss2:7.3311 exploreP:0.2177
Episode:956 meanR:10.6600 R:10.0000 rate:0.0200 aloss:0.7497 eloss:0.4424 aloss2:7.3755 exploreP:0.2175
Episode:957 meanR:10.6200 R:8.0000 rate:0.0160 aloss:0.7632 eloss:0.5065 aloss2:7.2341 exploreP:0.2174
Episode:958 meanR:10.6300 R:12.0000 rate:0.0240 aloss:0.7534 eloss:0.4050 aloss2:7.2922 exploreP:0.2171
Episode:959 meanR:10.5800 R:8.0000 rate:0.0160 aloss:0.7037 eloss:0.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
